## 1. Setup & Installation

First, let's install all required dependencies.

In [1]:
# Install required packages
# langgraph: The graph-based agent framework
# langchain-openai: OpenAI integration for LangChain
# langchain: Core LangChain library
# langchain-core: Core abstractions for LangChain

!pip install -q langgraph langchain-openai langchain langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.2 MB/s eta 0:00:00


## 2. API Key Configuration

To use OpenAI API, you need an API key from [OpenAI Platform](https://platform.openai.com/api-keys).

**Security Best Practice**: Never hardcode API keys in your code. Use environment variables or a secure configuration method.

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

Enter your OpenAI API Key: ··········
Enter your OpenAI Base URL: ··········


## 3. Model Configuration

Let's create a reusable model configuration using OpenAI's gpt-4o-mini model.

In [3]:
from langchain_openai import ChatOpenAI

# Configure the gpt-4.1-mini model via OpenAI
def get_model(temperature=0.7):
    """
    Create and return a configured gpt-4.1-mini model via OpenAI.

    Args:
        temperature (float): Controls randomness (0.0 = deterministic, 1.0 = creative)

    Returns:
        ChatOpenAI: Configured model instance
    """
    return ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=temperature,
        max_tokens=2048,
        base_url=os.environ.get("OPENAI_BASE_URL"),
    )

# Test the model
test_model = get_model()
response = test_model.invoke("Hello! Can you confirm you're working?")
print(f"Model Response: {response.content}")
print(f"\n✅ Model configured and tested successfully!")
print(f"📋 Base URL: {os.environ.get('OPENAI_BASE_URL')}")

Model Response: Hello! Yes, I'm here and ready to help. How can I assist you today?

✅ Model configured and tested successfully!
📋 Base URL: https://ds-ai-internship.openai.azure.com/openai/v1/


## 4. Import Required Libraries

Let's import all the necessary libraries for building our agentic patterns.

In [4]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
import operator

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


---

# Agentic Patterns

Now let's explore each agentic pattern with conceptual explanations and working code examples.

---

## Pattern A: Prompt Chaining

### Concept

**Prompt Chaining** is a sequential workflow pattern where the output of one agent becomes the input of the next agent. This creates a pipeline of transformations.

**Use Cases:**
- Multi-step content generation (outline → draft → polish)
- Data processing pipelines (extract → transform → analyze)
- Sequential reasoning tasks

**Flow:**
```
Input → Agent A → Intermediate Result → Agent B → Final Output
```

### Implementation

In [5]:
# Define the state structure for prompt chaining
class ChainState(TypedDict):
    topic: str
    outline: str
    article: str

# Agent A: Creates an outline
def create_outline(state: ChainState) -> ChainState:
    """
    Agent A: Generates a structured outline for the given topic.
    """
    print("🔵 Agent A: Creating outline...")

    model = get_model(temperature=0.7)
    prompt = f"Create a brief 3-point outline for an article about: {state['topic']}"

    response = model.invoke(prompt)
    state["outline"] = response.content

    print(f"✅ Outline created:\n{state['outline']}\n")
    return state

# Agent B: Expands the outline into a full article
def write_article(state: ChainState) -> ChainState:
    """
    Agent B: Takes the outline and writes a complete article.
    """
    print("🟢 Agent B: Writing article from outline...")

    model = get_model(temperature=0.8)
    prompt = f"""Based on this outline, write a short article (3-4 paragraphs):

Outline:
{state['outline']}

Article:"""

    response = model.invoke(prompt)
    state["article"] = response.content

    print(f"✅ Article written!\n")
    return state

# Build the graph
workflow_chain = StateGraph(ChainState)

# Add nodes (agents)
workflow_chain.add_node("outline_agent", create_outline)
workflow_chain.add_node("writer_agent", write_article)

# Define the flow: START → outline_agent → writer_agent → END
workflow_chain.add_edge(START, "outline_agent")
workflow_chain.add_edge("outline_agent", "writer_agent")
workflow_chain.add_edge("writer_agent", END)

# Compile the graph
chain_app = workflow_chain.compile()

print("✅ Prompt Chaining workflow created!")

✅ Prompt Chaining workflow created!


In [6]:
# Test the Prompt Chaining pattern
print("=" * 60)
print("Testing Prompt Chaining Pattern")
print("=" * 60)

result = chain_app.invoke({
    "topic": "The Future of Artificial Intelligence",
    "outline": "",
    "article": ""
})

print("\n" + "=" * 60)
print("FINAL ARTICLE:")
print("=" * 60)
print(result["article"])

Testing Prompt Chaining Pattern
🔵 Agent A: Creating outline...
✅ Outline created:
1. Emerging Trends and Technologies in AI  
2. Potential Societal and Economic Impacts  
3. Ethical Considerations and Regulatory Challenges

🟢 Agent B: Writing article from outline...
✅ Article written!


FINAL ARTICLE:
Artificial intelligence (AI) continues to evolve at a rapid pace, with emerging trends and technologies reshaping industries and everyday life. Advances in natural language processing, computer vision, and autonomous systems are enabling machines to perform increasingly complex tasks with greater accuracy and efficiency. Innovations such as generative AI models, reinforcement learning, and edge AI are expanding the possibilities for personalized experiences, real-time decision-making, and smart automation. These technological breakthroughs promise to revolutionize sectors ranging from healthcare and finance to transportation and education, fostering a new era of digital transformation.

T

## Pattern B: Routing

### Concept

**Routing** is a pattern where a router agent analyzes the input and dynamically decides which specialized agent or tool to invoke based on the query type.

**Use Cases:**
- Customer service chatbots (route to billing, technical, or sales)
- Multi-domain question answering
- Task classification and delegation

**Flow:**
```
                    ┌──> Specialist Agent A
Input → Router Agent ┼──> Specialist Agent B
                    └──> Specialist Agent C
```

### Implementation

In [7]:
# Define the state structure for routing
class RouteState(TypedDict):
    query: str
    route: str
    response: str

# Router Agent: Determines which specialist to call
def route_query(state: RouteState) -> RouteState:
    """
    Router Agent: Analyzes the query and determines the appropriate route.
    Routes: 'technical', 'creative', 'general'
    """
    print("🔀 Router Agent: Analyzing query...")

    model = get_model(temperature=0.3)
    prompt = f"""Classify this query into one category: 'technical', 'creative', or 'general'.
Respond with ONLY the category name.

Query: {state['query']}

Category:"""

    response = model.invoke(prompt)
    route = response.content.strip().lower()

    # Validate route
    if route not in ['technical', 'creative', 'general']:
        route = 'general'

    state["route"] = route
    print(f"✅ Query routed to: {route}\n")
    return state

# Technical Specialist Agent
def technical_agent(state: RouteState) -> RouteState:
    """
    Technical Specialist: Handles technical and scientific queries.
    """
    print("🔧 Technical Agent: Processing query...")

    model = get_model(temperature=0.5)
    prompt = f"""As a technical expert, provide a precise and detailed answer:

{state['query']}"""

    response = model.invoke(prompt)
    state["response"] = f"[Technical Agent] {response.content}"
    return state

# Creative Specialist Agent
def creative_agent(state: RouteState) -> RouteState:
    """
    Creative Specialist: Handles creative and artistic queries.
    """
    print("🎨 Creative Agent: Processing query...")

    model = get_model(temperature=0.9)
    prompt = f"""As a creative writer, provide an imaginative and engaging answer:

{state['query']}"""

    response = model.invoke(prompt)
    state["response"] = f"[Creative Agent] {response.content}"
    return state

# General Specialist Agent
def general_agent(state: RouteState) -> RouteState:
    """
    General Specialist: Handles general queries.
    """
    print("📝 General Agent: Processing query...")

    model = get_model(temperature=0.7)
    prompt = f"""Provide a helpful and balanced answer to this query:

{state['query']}"""

    response = model.invoke(prompt)
    state["response"] = f"[General Agent] {response.content}"
    return state

# Conditional routing function
def route_to_specialist(state: RouteState) -> str:
    """
    Returns the name of the next node based on the route.
    """
    route = state["route"]
    if route == "technical":
        return "technical_specialist"
    elif route == "creative":
        return "creative_specialist"
    else:
        return "general_specialist"

# Build the routing graph
workflow_route = StateGraph(RouteState)

# Add nodes
workflow_route.add_node("router", route_query)
workflow_route.add_node("technical_specialist", technical_agent)
workflow_route.add_node("creative_specialist", creative_agent)
workflow_route.add_node("general_specialist", general_agent)

# Define the flow
workflow_route.add_edge(START, "router")
workflow_route.add_conditional_edges(
    "router",
    route_to_specialist,
    {
        "technical_specialist": "technical_specialist",
        "creative_specialist": "creative_specialist",
        "general_specialist": "general_specialist"
    }
)
workflow_route.add_edge("technical_specialist", END)
workflow_route.add_edge("creative_specialist", END)
workflow_route.add_edge("general_specialist", END)

# Compile the graph
route_app = workflow_route.compile()

print("✅ Routing workflow created!")

✅ Routing workflow created!


In [8]:
# Test the Routing pattern with different query types
test_queries = [
    "Explain how neural networks learn through backpropagation",
    "Write a short poem about the stars",
    "What's the best way to stay productive?"
]

for query in test_queries:
    print("\n" + "=" * 60)
    print(f"Query: {query}")
    print("=" * 60)

    result = route_app.invoke({
        "query": query,
        "route": "",
        "response": ""
    })

    print(f"\n{result['response']}")


Query: Explain how neural networks learn through backpropagation
🔀 Router Agent: Analyzing query...
✅ Query routed to: technical

🔧 Technical Agent: Processing query...

[Technical Agent] Certainly! Here's a precise and detailed explanation of how neural networks learn through **backpropagation**:

---

### Overview

Backpropagation (short for **backward propagation of errors**) is a supervised learning algorithm used to train artificial neural networks. It efficiently computes the gradient of the loss function with respect to each weight in the network, enabling the use of gradient-based optimization methods (like gradient descent) to update the weights and minimize prediction error.

---

### Components Involved

1. **Neural Network Structure:**
   - Composed of layers: input layer, one or more hidden layers, and an output layer.
   - Each layer consists of neurons (units) with associated weights and biases.
   - Each neuron applies a weighted sum of its inputs followed by a nonline

## Pattern C: Parallelization

### Concept

**Parallelization** runs multiple agents or tasks simultaneously and then aggregates their results. This significantly improves performance for independent tasks.

**Use Cases:**
- Multi-perspective analysis
- Gathering information from multiple sources
- A/B testing different approaches

**Flow:**
```
         ┌──> Agent A ──┐
Input ───┼──> Agent B ──┼──> Aggregator → Output
         └──> Agent C ──┘
```

### Implementation

In [9]:
# Define the state structure for parallelization
# Using Annotated with operator.add to safely handle concurrent updates
class ParallelState(TypedDict):
    topic: str
    perspective_1: str  # Technical perspective
    perspective_2: str  # Business perspective
    perspective_3: str  # Ethical perspective
    summary: str
    # Track which perspectives are complete
    completed: Annotated[list[str], operator.add]

# Parallel Agent 1: Technical Perspective
def technical_perspective(state: ParallelState) -> ParallelState:
    """
    Analyzes the topic from a technical perspective.
    """
    print("🔧 Agent 1: Analyzing technical aspects...")

    model = get_model(temperature=0.6)
    prompt = f"""Analyze this topic from a TECHNICAL perspective (2-3 sentences):

{state['topic']}"""

    response = model.invoke(prompt)
    print("✅ Technical analysis complete")
    return {
        "perspective_1": response.content,
        "completed": ["technical"]
    }

# Parallel Agent 2: Business Perspective
def business_perspective(state: ParallelState) -> ParallelState:
    """
    Analyzes the topic from a business perspective.
    """
    print("💼 Agent 2: Analyzing business aspects...")

    model = get_model(temperature=0.6)
    prompt = f"""Analyze this topic from a BUSINESS perspective (2-3 sentences):

{state['topic']}"""

    response = model.invoke(prompt)
    print("✅ Business analysis complete")
    return {
        "perspective_2": response.content,
        "completed": ["business"]
    }

# Parallel Agent 3: Ethical Perspective
def ethical_perspective(state: ParallelState) -> ParallelState:
    """
    Analyzes the topic from an ethical perspective.
    """
    print("⚖️ Agent 3: Analyzing ethical aspects...")

    model = get_model(temperature=0.6)
    prompt = f"""Analyze this topic from an ETHICAL perspective (2-3 sentences):

{state['topic']}"""

    response = model.invoke(prompt)
    print("✅ Ethical analysis complete")
    return {
        "perspective_3": response.content,
        "completed": ["ethical"]
    }

# Aggregator Agent: Combines all perspectives
def aggregate_perspectives(state: ParallelState) -> ParallelState:
    """
    Aggregates all parallel analyses into a comprehensive summary.
    """
    print("\n🔄 Aggregator: Combining all perspectives...")

    model = get_model(temperature=0.7)
    prompt = f"""Create a comprehensive summary by combining these three perspectives:

Technical: {state['perspective_1']}

Business: {state['perspective_2']}

Ethical: {state['perspective_3']}

Summary (3-4 sentences):"""

    response = model.invoke(prompt)
    state["summary"] = response.content
    print("✅ Summary created")
    return state

# Build the parallel graph
workflow_parallel = StateGraph(ParallelState)

# Add nodes
workflow_parallel.add_node("tech_agent", technical_perspective)
workflow_parallel.add_node("business_agent", business_perspective)
workflow_parallel.add_node("ethics_agent", ethical_perspective)
workflow_parallel.add_node("aggregator", aggregate_perspectives)

# Define parallel execution: All three agents run simultaneously
workflow_parallel.add_edge(START, "tech_agent")
workflow_parallel.add_edge(START, "business_agent")
workflow_parallel.add_edge(START, "ethics_agent")

# All agents feed into the aggregator
workflow_parallel.add_edge("tech_agent", "aggregator")
workflow_parallel.add_edge("business_agent", "aggregator")
workflow_parallel.add_edge("ethics_agent", "aggregator")

workflow_parallel.add_edge("aggregator", END)

# Compile the graph
parallel_app = workflow_parallel.compile()

print("✅ Parallelization workflow created!")

✅ Parallelization workflow created!


In [10]:
# Test the Parallelization pattern
print("=" * 60)
print("Testing Parallelization Pattern")
print("=" * 60)

result = parallel_app.invoke({
    "topic": "Self-driving cars",
    "perspective_1": "",
    "perspective_2": "",
    "perspective_3": "",
    "summary": "",
    "completed": []
})

print("\n" + "=" * 60)
print("RESULTS FROM PARALLEL EXECUTION:")
print("=" * 60)
print(f"\n📊 Technical Perspective:\n{result['perspective_1']}")
print(f"\n💼 Business Perspective:\n{result['perspective_2']}")
print(f"\n⚖️ Ethical Perspective:\n{result['perspective_3']}")
print(f"\n🎯 Comprehensive Summary:\n{result['summary']}")

Testing Parallelization Pattern
💼 Agent 2: Analyzing business aspects...
⚖️ Agent 3: Analyzing ethical aspects...
🔧 Agent 1: Analyzing technical aspects...
✅ Ethical analysis complete
✅ Technical analysis complete
✅ Business analysis complete

🔄 Aggregator: Combining all perspectives...
✅ Summary created

RESULTS FROM PARALLEL EXECUTION:

📊 Technical Perspective:
Self-driving cars rely on a combination of sensors such as LiDAR, radar, and cameras to perceive their environment, coupled with advanced machine learning algorithms for object detection, localization, and decision-making. These systems integrate real-time data processing, sensor fusion, and control mechanisms to navigate complex traffic scenarios while ensuring safety and compliance with traffic regulations.

💼 Business Perspective:
From a business perspective, self-driving cars present significant opportunities for innovation and disruption across multiple industries, including automotive manufacturing, ride-sharing, and log

## Pattern D: Orchestrator-Worker

### Concept

**Orchestrator-Worker** (also called Manager-Worker) is a hierarchical pattern where a manager agent breaks down complex tasks into subtasks and delegates them to worker agents in a coordinated manner.

**Use Cases:**
- Complex project planning and execution
- Multi-step research tasks
- Iterative problem-solving workflows

**Flow:**
```
Manager ──> Step 1 ──> Worker ──┐
   │                            │
   └──> Step 2 ──> Worker ──────┤
   │                            ├──> Final Result
   └──> Step 3 ──> Worker ──────┘
```

### Implementation

In [11]:
# Define the state structure for orchestrator-worker
class OrchestratorState(TypedDict):
    task: str
    plan: list[str]
    current_step: int
    results: list[str]
    final_report: str

# Manager Agent: Creates the plan
def manager_agent(state: OrchestratorState) -> OrchestratorState:
    """
    Manager Agent: Breaks down the complex task into 3 actionable steps.
    """
    print("👔 Manager Agent: Creating execution plan...")

    model = get_model(temperature=0.5)
    prompt = f"""Break down this task into exactly 3 clear, actionable steps.
Format: Return only 3 lines, each starting with a number.

Task: {state['task']}

Steps:"""

    response = model.invoke(prompt)
    # Parse the steps
    steps = [line.strip() for line in response.content.split('\n') if line.strip() and line.strip()[0].isdigit()]
    state["plan"] = steps[:3]  # Ensure we have exactly 3 steps
    state["current_step"] = 0
    state["results"] = []

    print(f"✅ Plan created with {len(state['plan'])} steps")
    for i, step in enumerate(state['plan'], 1):
        print(f"   Step {i}: {step}")
    return state

# Worker Agent: Executes individual steps
def worker_agent(state: OrchestratorState) -> OrchestratorState:
    """
    Worker Agent: Executes the current step from the plan.
    """
    current = state["current_step"]
    step = state["plan"][current]

    print(f"\n👷 Worker Agent: Executing step {current + 1}...")
    print(f"   Task: {step}")

    model = get_model(temperature=0.7)
    prompt = f"""Execute this step and provide the result (2-3 sentences):

Step: {step}
Context: This is part of the larger task: {state['task']}

Result:"""

    response = model.invoke(prompt)
    state["results"].append(response.content)
    state["current_step"] += 1

    print(f"✅ Step {current + 1} completed")
    return state

# Decide whether to continue or finish
def should_continue(state: OrchestratorState) -> str:
    """
    Determines if there are more steps to execute.
    """
    if state["current_step"] < len(state["plan"]):
        return "worker"  # More steps to execute
    else:
        return "finalizer"  # All steps completed

# Finalizer Agent: Compiles results
def finalizer_agent(state: OrchestratorState) -> OrchestratorState:
    """
    Finalizer Agent: Combines all worker results into a final report.
    """
    print("\n📋 Finalizer Agent: Creating final report...")

    model = get_model(temperature=0.6)

    # Compile all results
    results_text = "\n\n".join([f"Step {i+1} Result: {result}"
                                 for i, result in enumerate(state["results"])])

    prompt = f"""Create a comprehensive final report by synthesizing these step results:

{results_text}

Original Task: {state['task']}

Final Report (3-4 sentences):"""

    response = model.invoke(prompt)
    state["final_report"] = response.content

    print("✅ Final report created")
    return state

# Build the orchestrator-worker graph
workflow_orchestrator = StateGraph(OrchestratorState)

# Add nodes
workflow_orchestrator.add_node("manager", manager_agent)
workflow_orchestrator.add_node("worker", worker_agent)
workflow_orchestrator.add_node("finalizer", finalizer_agent)

# Define the flow
workflow_orchestrator.add_edge(START, "manager")
workflow_orchestrator.add_conditional_edges(
    "manager",
    should_continue,
    {
        "worker": "worker",
        "finalizer": "finalizer"
    }
)
# Worker can loop back to itself or go to finalizer
workflow_orchestrator.add_conditional_edges(
    "worker",
    should_continue,
    {
        "worker": "worker",
        "finalizer": "finalizer"
    }
)
workflow_orchestrator.add_edge("finalizer", END)

# Compile the graph
orchestrator_app = workflow_orchestrator.compile()

print("✅ Orchestrator-Worker workflow created!")

✅ Orchestrator-Worker workflow created!


In [12]:
# Test the Orchestrator-Worker pattern
print("=" * 60)
print("Testing Orchestrator-Worker Pattern")
print("=" * 60)

result = orchestrator_app.invoke({
    "task": "Plan and execute a social media marketing campaign for a new mobile app",
    "plan": [],
    "current_step": 0,
    "results": [],
    "final_report": ""
})

print("\n" + "=" * 60)
print("EXECUTION RESULTS:")
print("=" * 60)

for i, step_result in enumerate(result["results"], 1):
    print(f"\n📌 Step {i} Result:")
    print(step_result)

print("\n" + "=" * 60)
print("FINAL REPORT:")
print("=" * 60)
print(result["final_report"])

Testing Orchestrator-Worker Pattern
👔 Manager Agent: Creating execution plan...
✅ Plan created with 3 steps
   Step 1: 1. Research target audience and define campaign goals and key messages.
   Step 2: 2. Create engaging content and schedule posts across selected social media platforms.
   Step 3: 3. Launch the campaign, monitor performance, and adjust strategies based on analytics.

👷 Worker Agent: Executing step 1...
   Task: 1. Research target audience and define campaign goals and key messages.
✅ Step 1 completed

👷 Worker Agent: Executing step 2...
   Task: 2. Create engaging content and schedule posts across selected social media platforms.
✅ Step 2 completed

👷 Worker Agent: Executing step 3...
   Task: 3. Launch the campaign, monitor performance, and adjust strategies based on analytics.
✅ Step 3 completed

📋 Finalizer Agent: Creating final report...
✅ Final report created

EXECUTION RESULTS:

📌 Step 1 Result:
The target audience for the new mobile app consists primarily of tec

## Pattern E: Evaluator-Optimizer

### Concept

**Evaluator-Optimizer** implements a feedback loop where one agent generates a solution, another agent evaluates it and provides critique, and the generator improves the solution based on the feedback.

**Use Cases:**
- Code review and improvement
- Content refinement
- Iterative problem-solving
- Quality assurance workflows

**Flow:**
```
Generator ──> Draft ──> Evaluator ──> Critique
    ↑                                    │
    └────────── Improvement ─────────────┘
```

### Implementation

In [13]:
# Define the state structure for evaluator-optimizer
class EvaluatorState(TypedDict):
    task: str
    solution: str
    critique: str
    iteration: int
    max_iterations: int
    is_approved: bool

# Generator Agent: Creates or improves the solution
def generator_agent(state: EvaluatorState) -> EvaluatorState:
    """
    Generator Agent: Creates initial solution or improves based on critique.
    """
    iteration = state["iteration"]

    if iteration == 0:
        print("🎨 Generator Agent: Creating initial solution...")
        model = get_model(temperature=0.8)
        prompt = f"""Create a solution for this task (keep it short, 2-3 sentences):

{state['task']}

Solution:"""
    else:
        print(f"🔧 Generator Agent: Improving solution (Iteration {iteration})...")
        model = get_model(temperature=0.7)
        prompt = f"""Improve this solution based on the critique:

Original Solution: {state['solution']}

Critique: {state['critique']}

Improved Solution (keep it short, 2-3 sentences):"""

    response = model.invoke(prompt)
    state["solution"] = response.content
    state["iteration"] += 1

    print(f"✅ Solution {'created' if iteration == 0 else 'improved'}")
    return state

# Evaluator Agent: Reviews and critiques the solution
def evaluator_agent(state: EvaluatorState) -> EvaluatorState:
    """
    Evaluator Agent: Reviews the solution and provides critique.
    """
    print(f"\n🔍 Evaluator Agent: Reviewing solution...")

    model = get_model(temperature=0.5)
    prompt = f"""Evaluate this solution and provide feedback.

Task: {state['task']}
Solution: {state['solution']}

Provide your evaluation:
1. Is it good enough? (YES/NO)
2. If NO, what specific improvements are needed?

Start your response with 'APPROVED' if it's good enough, or 'NEEDS IMPROVEMENT' if not.

Evaluation:"""

    response = model.invoke(prompt)
    critique = response.content
    state["critique"] = critique

    # Check if approved
    if "APPROVED" in critique.upper().split('\n')[0]:
        state["is_approved"] = True
        print("✅ Solution APPROVED!")
    else:
        state["is_approved"] = False
        print("❌ Needs improvement")

    return state

# Decide whether to continue improving or finish
def should_continue_optimization(state: EvaluatorState) -> str:
    """
    Determines if we should continue iterating or finish.
    """
    # Stop if approved or reached max iterations
    if state["is_approved"]:
        return "end"
    elif state["iteration"] >= state["max_iterations"]:
        print(f"\n⚠️ Max iterations ({state['max_iterations']}) reached")
        return "end"
    else:
        return "generator"  # Continue improving

# Build the evaluator-optimizer graph
workflow_evaluator = StateGraph(EvaluatorState)

# Add nodes
workflow_evaluator.add_node("generator", generator_agent)
workflow_evaluator.add_node("evaluator", evaluator_agent)

# Define the flow
workflow_evaluator.add_edge(START, "generator")
workflow_evaluator.add_edge("generator", "evaluator")
workflow_evaluator.add_conditional_edges(
    "evaluator",
    should_continue_optimization,
    {
        "generator": "generator",  # Loop back for improvement
        "end": END
    }
)

# Compile the graph
evaluator_app = workflow_evaluator.compile()

print("✅ Evaluator-Optimizer workflow created!")

✅ Evaluator-Optimizer workflow created!


In [14]:
# Test the Evaluator-Optimizer pattern
print("=" * 60)
print("Testing Evaluator-Optimizer Pattern")
print("=" * 60)

result = evaluator_app.invoke({
    "task": "Write a compelling tagline for an eco-friendly water bottle company",
    "solution": "",
    "critique": "",
    "iteration": 0,
    "max_iterations": 3,
    "is_approved": False
})

print("\n" + "=" * 60)
print("OPTIMIZATION RESULTS:")
print("=" * 60)
print(f"\nTotal Iterations: {result['iteration']}")
print(f"Status: {'✅ APPROVED' if result['is_approved'] else '⚠️ Max iterations reached'}")
print(f"\n🎯 Final Solution:\n{result['solution']}")
print(f"\n📝 Final Evaluation:\n{result['critique']}")

Testing Evaluator-Optimizer Pattern
🎨 Generator Agent: Creating initial solution...
✅ Solution created

🔍 Evaluator Agent: Reviewing solution...
✅ Solution APPROVED!

OPTIMIZATION RESULTS:

Total Iterations: 1
Status: ✅ APPROVED

🎯 Final Solution:
Stay refreshed, save the planet — sip sustainably with our eco-friendly bottles.

📝 Final Evaluation:
APPROVED

The tagline is clear, engaging, and effectively communicates both the product benefit ("Stay refreshed") and the eco-friendly mission ("save the planet," "sip sustainably"). It uses a nice rhythm and alliteration that make it memorable. Overall, it is compelling and appropriate for an eco-friendly water bottle company.


---

# Conclusion

## Pattern Selection Guide

Choosing the right agentic pattern depends on your specific use case:

### When to Use Each Pattern

| Pattern | Best For | Key Benefit |
|---------|----------|-------------|
| **Prompt Chaining** | Sequential transformations, multi-stage content generation | Simplicity and clarity |
| **Routing** | Multi-domain problems, task classification | Specialization and accuracy |
| **Parallelization** | Independent tasks, multi-perspective analysis | Speed and efficiency |
| **Orchestrator-Worker** | Complex planning, multi-step execution | Coordination and structure |
| **Evaluator-Optimizer** | Quality-critical tasks, iterative refinement | Quality and improvement |

### Combining Patterns

These patterns can be combined for even more powerful workflows:

- **Routing + Prompt Chaining**: Route to specialized chains
- **Orchestrator + Parallelization**: Manager delegates parallel tasks
- **Parallelization + Evaluator**: Multiple solutions evaluated in parallel
- **All patterns together**: Complex enterprise systems

### Key Takeaways

1. **Start Simple**: Begin with basic patterns and add complexity as needed
2. **State Management**: Langgraph's state management is crucial for complex workflows
3. **Error Handling**: Always implement proper error handling in production
4. **Testing**: Test each agent independently before combining
5. **Monitoring**: Track agent performance and iterations
6. **Cost Management**: Be mindful of API calls in loops and parallel operations

### Next Steps

- Experiment with different combinations of patterns
- Add memory and persistence to your agents
- Integrate external tools and APIs
- Build domain-specific multi-agent systems
- Implement proper logging and monitoring

## Resources

- [Langgraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangChain Documentation](https://python.langchain.com/)

---

**Thank you for completing this tutorial!** 🎉

Feel free to adapt these patterns for your own projects. Happy building! 🚀